In [1]:
#   @@@@@@@     @      @    @  @   #
#   @  @  @    @ @      @  @       #
#      @  @   @   @      @@    @   #
#      @     @@@@@@@    @  @   @   #
#      @    @       @  @    @  @   #

Привет, в этой практике мы с вами применим наши знания по PySpark и постараемся изучить что-то новое в процессе выполнения.
<br>В занятии используется датасет собранный на основе данных <a href="https://www.kaggle.com/chicago/chicago-taxi-rides-2016">Chicago Taxi Rides 2016</a>
<br>Полная <a href="https://spark.apache.org/docs/latest/api/python/index.html">документация PySpark</a>.
<br>Схема данны:
<br>|-- taxi_id = идентификатор таксиста
<br>|-- trip_start_timestamp = время начала поездки
<br>|-- trip_end_timestamp = время окончания поездки
<br>|-- trip_seconds = время длительности поездки в секундах
<br>|-- trip_miles = мили проиденные во время поездки
<br>|-- fare = транспортные расходы
<br>|-- tips = назначенные чаевые
<br>|-- trip_total = общая стоимость поездки (Итоговая с учетом чаевых и расходов)
<br>|-- payment_type = тип оплаты

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, max, min, stddev

In [3]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

In [4]:
spark.conf.set("spark.sql.session.timeZone", "GMT+3")

In [5]:
spark

Скачайте <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_data.parquet">taxi_data.parquet</a> и загрузите используя <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">SparkAPI</a>

In [6]:
df = spark.read.parquet('taxi_data.parquet')

№1 Посчитайте количество загруженных строк.

In [7]:
df.count() # Число строк

2540712

In [8]:
df.show(5)

+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare|tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|   5240| 2016-12-15 23:45:00|2016-12-16 00:00:00|         900|       2.5|10.75|2.45|      14.7| Credit Card|
|   1215| 2016-12-12 07:15:00|2016-12-12 07:15:00|         240|       0.4|  5.0| 3.0|       9.5| Credit Card|
|   3673| 2016-12-16 16:30:00|2016-12-16 17:00:00|        2400|      10.7| 31.0| 0.0|      31.0|        Cash|
|   5400| 2016-12-16 08:45:00|2016-12-16 09:00:00|         300|       0.0| 5.25| 2.0|      7.25| Credit Card|
|   1257| 2016-12-03 18:45:00|2016-12-03 18:45:00|         360|       0.3|  5.0| 0.0|       5.0|        Cash|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
only showi

Посмотрим схему данных:

In [9]:
df.printSchema()

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



№2 Чему равна корреляция и ковариация между длиной маршрута и ценой за поездку? Ответ округлите до 5 знаков после запятой.
<br>Подробнее <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.corr.html?highlight=corr#pyspark-sql-dataframe-corr">corr</a> & <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.cov.html?highlight=cov#pyspark-sql-dataframe-cov">cov</a>

In [10]:
round(df.corr('trip_miles', 'trip_total'), 5)

0.44816

In [11]:
round(df.cov('trip_miles', 'trip_total'), 5)

71.96914

№3 Найдите количество, среднее, cреднеквадратическое отклонение, минимум и максимум для длины маршрута и цены за поездку? Ответ округлите до 1 знака после запятой. Подробнее <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.describe.html?highlight=describe#pyspark-sql-dataframe-describe">describe</a>

In [12]:
df.describe(['trip_miles', 'trip_total']).show()

+-------+------------------+------------------+
|summary|        trip_miles|        trip_total|
+-------+------------------+------------------+
|  count|           2540677|           2540672|
|   mean|3.0005873828090266|15.913560215564042|
| stddev|  5.25716922943536|30.546699217618237|
|    min|               0.0|               0.0|
|    max|             900.0|           9276.69|
+-------+------------------+------------------+



округленные количества значений колонок

In [13]:
round(df.select(count(col('trip_miles'))).collect()[0][0], 1)

2540677

In [14]:
round(df.select(count(col('trip_total'))).collect()[0][0], 1)

2540672

округленные средние значения колонок

In [15]:
round(df.select(avg(col('trip_miles'))).collect()[0][0], 1)

3.0

In [16]:
round(df.select(avg(col('trip_total'))).collect()[0][0], 1)

15.9

округленные стандартные отклонения

In [17]:
round(df.select(stddev(col('trip_miles'))).collect()[0][0], 1)

5.3

In [18]:
round(df.select(stddev(col('trip_total'))).collect()[0][0], 1)

30.5

округленные минимальные значения

In [19]:
round(df.select(min(col('trip_miles'))).collect()[0][0], 1)

0.0

In [20]:
round(df.select(min(col('trip_total'))).collect()[0][0], 1)

0.0

округленные максимальные значения

In [21]:
round(df.select(max(col('trip_miles'))).collect()[0][0], 1)

900.0

In [22]:
round(df.select(max(col('trip_total'))).collect()[0][0], 1)

9276.7

№4 Найдите самый НЕпопулярный вид оплаты.
<br>Подробнее <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.groupBy.html?highlight=groupby#pyspark-sql-dataframe-groupby">groupBy</a> <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.orderBy.html?highlight=orderby#pyspark-sql-dataframe-orderby">orderBy</a>

In [23]:
df.groupBy('payment_type').count().orderBy('count', ascending=True).collect()[0][0]

'Way2ride'

№5 Найдите идентификатор таксиста выполнившего наибольшее число заказов.

In [24]:
df.groupBy('taxi_id').count().orderBy('count', ascending=False).collect()[0][0]

316

№6 Чему равна средняя цена среди поездок, оплаченных наличными? Ответ округлите до 5 знака.
<br> Подробней <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.where.html?highlight=where#pyspark-sql-dataframe-where">where</a>

In [25]:
df_cash = df.where(col('payment_type') == 'Cash')

In [26]:
round(df_cash.select(avg(col('trip_total'))).collect()[0][0], 5)

12.03526

№7 Сколько таксистов проехало больше 1000 миль за все время выполнения заказов?

In [27]:
df.groupBy('taxi_id').sum('trip_miles').where('sum(trip_miles) > 1000').count()

2860

№8 Сколько миль проехал пассажир в самой долгой поездке? (Ответ округлите до целого)

In [28]:
round(df.orderBy('trip_seconds', ascending=False).where(col('trip_seconds')\
                                                        .isNull() == False).select('trip_miles').collect()[0][0])

0

№9 Каков средний заработок всех таксистов? Ответ округлите до 5-ого знака.
<br>Отсеките неизвестные машины (не определенный taxi_id).

In [29]:
round(df.where(col('taxi_id').isNotNull()).groupBy('taxi_id').sum('trip_total')\
                          .select(avg(col('sum(trip_total)'))).collect()[0][0], 5)

8218.85627

№10 Сколько поездок начиналось в самый загруженный час?
<br>Используйте функцию <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.hour.html?highlight=hour#pyspark-sql-functions-hour">hour</a>

In [30]:
from pyspark.sql.functions import hour

In [31]:
df.select(hour('trip_start_timestamp').alias('hour')).groupBy('hour').count().orderBy('count', ascending=False).show(1)

+----+------+
|hour| count|
+----+------+
|  18|181127|
+----+------+
only showing top 1 row



№11 Сколько поездок началось во второй четверти суток?

In [32]:
df_hours = df.withColumn('hour', hour('trip_start_timestamp')).groupBy('hour').count()

In [33]:
df_hours.where((col('hour') > 5) & (col('hour') < 12)).agg({'count':'sum'}).show()

+----------+
|sum(count)|
+----------+
|    538737|
+----------+



№12 Найдите топ три даты, в которые было суммарно больше всего чаевых? (Чаевые выдаются после совершения поездки)
<br> Ожидаемый формат дат YYYY-MM-DD
<br>Вам может понадобится конвертация типов <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.Column.cast.html?highlight=cast#pyspark-sql-column-cast">cast</a>

In [34]:
from pyspark.sql.types import DateType

In [35]:
df.select(df['trip_end_timestamp'].cast('date').alias('date'), df['tips']).groupBy('date')\
                                    .agg({"tips":'sum'}).orderBy('sum(tips)', ascending=False).show(3)

+----------+------------------+
|      date|         sum(tips)|
+----------+------------------+
|2016-11-03|110102.37000000013|
|2016-11-09|106187.87999999986|
|2016-11-16| 99993.77000000038|
+----------+------------------+
only showing top 3 rows



№13 Сколько было заказов в дату с наибольшим спросом?

In [36]:
df.select(df['trip_start_timestamp'].cast('date').alias('date')).groupBy('date')\
                                    .count().orderBy('count', ascending=False).show(1)

+----------+-----+
|      date|count|
+----------+-----+
|2016-11-03|61259|
+----------+-----+
only showing top 1 row



Подгрузите данные о марках машин из датасета <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_cars_data.parquet">taxi_cars_data.parquet</a>

In [37]:
df_car = spark.read.parquet('taxi_cars_data.parquet')

In [38]:
df_car.show()

+-------+-------------------+
|taxi_id|          car_model|
+-------+-------------------+
|   1159|       Toyota Prius|
|   7273|Ford Crown Victoria|
|   2904|        Honda Civic|
|   3210|        Ford Fusion|
|   2088|       Toyota Camry|
|   4821|Ford Crown Victoria|
|   2069|    Hyundai Elantra|
|   6240|       Toyota Camry|
|    296|     Hyundai Sonata|
|   2136|Ford Crown Victoria|
|   1436|     Toyota Corolla|
|   1090|     Toyota Corolla|
|   7711|        Lincoln MKZ|
|   1572|Ford Crown Victoria|
|   3959|     Chevrolet Aveo|
|   5645|  Chevrolet Lacetti|
|   5149|            Audi A7|
|   6366|     Hyundai Sonata|
|    451|     Hyundai Accent|
|   1669|           Kia Ceed|
+-------+-------------------+
only showing top 20 rows



№14 Какая марка машины самая распрастранненая среди таксистов?
<br>Подробнее <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.split.html?highlight=split#pyspark-sql-functions-split">split</a>

In [39]:
from pyspark.sql.functions import split

In [40]:
df_car.withColumn('mark', split(col('car_model'), ' ')[0]).groupBy('mark').count().orderBy('count', ascending=False).show(1)

+----+-----+
|mark|count|
+----+-----+
|Ford| 1484|
+----+-----+
only showing top 1 row



№15 Сколько раз и какая модель машин чаще всего встречается в поездках?
<br>Подробнее <a href="https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.split.html?highlight=split#pyspark-sql-functions-split">join</a>

In [41]:
df.join(df_car, on='taxi_id').groupBy('car_model').count().orderBy('count', ascending=False).show(1)

+-------------------+------+
|          car_model| count|
+-------------------+------+
|Ford Crown Victoria|388682|
+-------------------+------+
only showing top 1 row



Почувствуй силу сжатия! сохрани DataFrame в csv и сравни размеры файлов.

In [42]:
df.coalesce(1).write.csv('taxi_data_csv')

Теперь загрузите данные из csv и проверьте типы методом printSchema().

In [43]:
df_csv = spark.read.csv('taxi_data_csv')
df_csv.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [44]:
df.printSchema()

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



Не забудьте посетить SparkUI и изучить историю ваших задач.

In [45]:
spark